In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [ ]:
import copy
import datetime
import json
import os
import scipy.spatial
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [ ]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
with open('./data/western_north_america/western_north_america_command.json', 'r') as f:
    command = json.load(f)
station = pd.read_csv(command["station_file_name"])
segment = pd.read_csv(command["segment_file_name"])
block = pd.read_csv(command["block_file_name"])

In [ ]:
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command)
segment, station, block = celeri.assign_block_labels(segment, station, block)

In [ ]:
plt.figure()
plt.title("West and east labels")
for i in range(len(segment)):
    plt.plot([segment.lon1.values[i], segment.lon2.values[i]],
             [segment.lat1.values[i], segment.lat2.values[i]],
             "-k", linewidth=0.5)
    plt.text(segment.mid_lon_plate_carree.values[i],
             segment.mid_lat_plate_carree.values[i],
             str(segment['west_labels'][i]) + "," + str(segment['east_labels'][i]),
             fontsize=8, color="m", horizontalalignment="center", verticalalignment="center")

for i in range(len(station)):
    plt.text(station.lon.values[i],
             station.lat.values[i],
             str(station.block_label[i]),
             fontsize=8, color="k", horizontalalignment="center", verticalalignment="center")

for i in range(len(block)):
    plt.text(block.interior_lon.values[i],
             block.interior_lat.values[i],
             str(block.block_label[i]),
             fontsize=8, color="g", horizontalalignment="center", verticalalignment="center")

plt.gca().set_aspect('equal')
plt.show()